# Lab 9: Smart Contract Security & DeFi Attack Analysis

Vulnerability analysis, protocol simulation, and MEV detection

> **Expected Time**
>
> -   FIN510: Exercises 1-2 ≈ 75 min
> -   FIN720: All exercises ≈ 110 min
> -   Directed learning extensions ≈ 60 min

<figure>
<a
href="https://colab.research.google.com/github/quinfer/fin510-colab-notebooks/blob/main/labs/lab09_smart_contracts.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
<figcaption>Open in Colab</figcaption>
</figure>

## Before You Code: The Big Picture

Smart contracts are **“code is law”**: once deployed, they execute
deterministically—bugs and all. DeFi protocols manage **\$50B+ in locked
value**, making them prime targets for hackers.

> **The DeFi Security Crisis**
>
> **The Promise:** 1. **Trustless**: No intermediaries, code enforces
> rules 2. **Composability**: DeFi “money legos” build on each other 3.
> **Transparency**: All code is open-source, auditable
>
> **The Reality:** - **\$3.1 billion stolen in DeFi hacks in 2022**
> (Chainalysis) - Average hack size: \$10-50M (multi-sig and oracle
> exploits) - Largest single exploit: Ronin Bridge (\$625M, 2022)
>
> **Top Vulnerabilities:** 1. **Reentrancy**: DAO hack (\$60M, 2016),
> Cream Finance (\$130M, 2021) 2. **Flash loans**: bZx (\$8M, 2020),
> Harvest Finance (\$34M, 2020) 3. **Oracle manipulation**: Mango
> Markets (\$110M, 2022) 4. **Access control**: Poly Network (\$611M,
> 2021—funds returned) 5. **MEV extraction**: \$676M extracted in 2021
> (Flashbots data)
>
> **The Dilemma:** - **Innovation speed** (ship fast, iterate)
> vs. **Security rigor** (audit, formal verification) - Most protocols
> launch with \<2 week audits (insufficient for \$100M TVL) - “Code is
> law” means irreversible losses (no chargebacks, no insurance for most)

### What You’ll Build Today

By the end of this lab, you will have:

-   ✅ Understanding of reentrancy attacks (and how to prevent them)
-   ✅ Simulation of flash loan attacks on lending protocols
-   ✅ MEV detection (sandwich attacks, frontrunning)
-   ✅ Critical perspective on DeFi security vs. usability tradeoffs

**Time estimate:** 75 minutes (FIN510) \| 110 minutes (FIN720 with all
exercises)

> **Why This Matters**
>
> Smart contract security is a \$500M+ industry (auditing firms, bug
> bounties, insurance). If you can read Solidity and understand exploit
> mechanics, you’re employable at Consensys, OpenZeppelin, Trail of
> Bits, or any DeFi protocol. This lab teaches you the attack patterns
> that matter.

## Learning Objectives

By the end of this lab, you will be able to:

-   Identify common smart contract vulnerabilities (reentrancy,
    overflow, access control)
-   Simulate exploit mechanics demonstrating how attacks work
-   Implement security fixes and verify their effectiveness
-   Model DeFi protocol economics (AMMs, flash loans, impermanent loss)
-   Simulate economic attacks on lending protocols
-   Detect MEV extraction patterns (sandwich attacks, arbitrage,
    liquidations)
-   Analyze MEV impact on ordinary users
-   Evaluate trade-offs between DeFi innovation and security

## Setup and Dependencies

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✓ Setup complete - ready for smart contract analysis")

## Exercise 1: Smart Contract Vulnerability Analysis

### Understanding Common Vulnerabilities

Smart contracts face unique security challenges due to immutability—once
deployed, bugs cannot be patched. We’ll analyze three critical
vulnerability classes that have cost billions in losses: reentrancy,
integer overflow, and access control failures.

### Reentrancy Vulnerability Simulation

In [2]:
# Simplified smart contract simulation in Python
class VulnerableBank:
    """Vulnerable withdrawal contract with reentrancy bug"""
    
    def __init__(self):
        self.balances = {}
        self.total_balance = 0
        self.transaction_log = []
        
    def deposit(self, user, amount):
        """Deposit funds"""
        if user not in self.balances:
            self.balances[user] = 0
        self.balances[user] += amount
        self.total_balance += amount
        self.transaction_log.append(f"{user} deposited {amount}")
        
    def withdraw(self, user, external_call_function=None):
        """
        Vulnerable withdrawal function
        VULNERABILITY: External call before state update
        """
        amount = self.balances.get(user, 0)
        
        if amount > 0:
            # VULNERABLE: External call before state update!
            # In real Ethereum, this would be: user.call{value: amount}("")
            if external_call_function:
                self.transaction_log.append(f"Calling external contract for {user}")
                external_call_function(self, user)  # Attacker can reenter here!
            
            # State update happens AFTER external call (too late!)
            self.balances[user] = 0
            self.total_balance -= amount
            self.transaction_log.append(f"{user} withdrew {amount}")
            return amount
        return 0
    
    def get_balance(self, user):
        return self.balances.get(user, 0)


class ReentrancyAttacker:
    """Attacker contract exploiting reentrancy"""
    
    def __init__(self):
        self.stolen = 0
        self.attack_count = 0
        
    def attack(self, vulnerable_bank, user):
        """Execute reentrancy attack"""
        initial_balance = vulnerable_bank.get_balance(user)
        
        # Trigger withdrawal which will call back to this contract
        withdrawn = vulnerable_bank.withdraw(user, self.reenter_callback)
        
        print(f"\n{'='*70}")
        print(f"REENTRANCY ATTACK SUMMARY")
        print(f"{'='*70}")
        print(f"Attacker initial balance: {initial_balance}")
        print(f"Times reentered: {self.attack_count}")
        print(f"Total stolen: {self.stolen}")
        print(f"Bank total balance after attack: {vulnerable_bank.total_balance}")
        
    def reenter_callback(self, bank, user):
        """
        Callback function - called during withdrawal
        Can reenter withdraw() before state is updated!
        """
        self.attack_count += 1
        
        # Check if we can still withdraw (balance hasn't been set to 0 yet!)
        if bank.get_balance(user) > 0 and self.attack_count < 5:  # Limit recursion
            self.stolen += bank.get_balance(user)
            # Reenter withdraw function!
            bank.withdraw(user, self.reenter_callback)


# Demonstrate reentrancy attack
print("="*70)
print("REENTRANCY VULNERABILITY DEMONSTRATION")
print("="*70)

# Setup
bank = VulnerableBank()
bank.deposit("Alice", 100)
bank.deposit("Bob", 100)
bank.deposit("Attacker", 100)

print("\nInitial state:")
print(f"  Alice balance: {bank.get_balance('Alice')}")
print(f"  Bob balance: {bank.get_balance('Bob')}")
print(f"  Attacker balance: {bank.get_balance('Attacker')}")
print(f"  Bank total: {bank.total_balance}")

# Execute attack
print("\n" + "-"*70)
print("Executing reentrancy attack...")
print("-"*70)

attacker = ReentrancyAttacker()
attacker.attack(bank, "Attacker")

print("\nFinal state:")
print(f"  Alice balance: {bank.get_balance('Alice')}")
print(f"  Bob balance: {bank.get_balance('Bob')}")
print(f"  Attacker balance: {bank.get_balance('Attacker')}")
print(f"  Bank total: {bank.total_balance}")

print("\n💀 Attack successful! Attacker drained more than their balance through reentrancy")

### Secure Implementation with Reentrancy Guard

In [3]:
class SecureBank:
    """Secure bank with reentrancy protection"""
    
    def __init__(self):
        self.balances = {}
        self.total_balance = 0
        self.transaction_log = []
        self.locked = False  # Reentrancy guard
        
    def deposit(self, user, amount):
        """Deposit funds"""
        if user not in self.balances:
            self.balances[user] = 0
        self.balances[user] += amount
        self.total_balance += amount
        
    def withdraw(self, user, external_call_function=None):
        """
        Secure withdrawal function
        FIX 1: Reentrancy guard (mutex lock)
        FIX 2: Checks-Effects-Interactions pattern
        """
        # Reentrancy guard
        if self.locked:
            print(f"  ⚠️ Reentrancy attempt blocked for {user}!")
            return 0
            
        self.locked = True  # Lock to prevent reentrancy
        
        try:
            amount = self.balances.get(user, 0)
            
            if amount > 0:
                # SECURE: Update state BEFORE external call
                self.balances[user] = 0
                self.total_balance -= amount
                self.transaction_log.append(f"{user} withdrew {amount}")
                
                # External call happens AFTER state update
                if external_call_function:
                    external_call_function(self, user)
                
                return amount
            return 0
        finally:
            self.locked = False  # Always unlock, even if exception occurs


# Demonstrate attack fails against secure contract
print("\n" + "="*70)
print("TESTING ATTACK AGAINST SECURE CONTRACT")
print("="*70)

secure_bank = SecureBank()
secure_bank.deposit("Alice", 100)
secure_bank.deposit("Bob", 100)
secure_bank.deposit("Attacker", 100)

print("\nInitial state:")
print(f"  Bank total: {secure_bank.total_balance}")

print("\n" + "-"*70)
print("Attempting reentrancy attack on secure contract...")
print("-"*70)

secure_attacker = ReentrancyAttacker()
secure_attacker.attack(secure_bank, "Attacker")

print("\nFinal state:")
print(f"  Bank total: {secure_bank.total_balance}")
print(f"  Attacker only stole: {secure_attacker.stolen}")

print("\n✅ Attack blocked! Reentrancy guard prevented exploitation")

### Integer Overflow Vulnerability

In [4]:
class TokenContractVulnerable:
    """Token contract vulnerable to overflow"""
    
    def __init__(self, supply=1000):
        self.balances = {"Owner": supply}
        self.MAX_UINT = 2**256 - 1  # Maximum 256-bit unsigned integer
        
    def transfer(self, from_addr, to_addr, amount):
        """
        Transfer tokens
        VULNERABILITY: No overflow checking
        """
        # Vulnerable subtraction (can underflow)
        if self.balances.get(from_addr, 0) >= amount:
            self.balances[from_addr] = self.balances.get(from_addr, 0) - amount
            
            # Vulnerable addition (can overflow, though less common)
            self.balances[to_addr] = self.balances.get(to_addr, 0) + amount
            return True
        return False
    
    def batch_transfer(self, from_addr, recipients):
        """
        Batch transfer - calculate total first
        VULNERABILITY: Total calculation can overflow!
        """
        total = 0
        for recipient, amount in recipients:
            total += amount  # Can overflow!
            
        # If total overflowed to small number, this check passes incorrectly
        if self.balances.get(from_addr, 0) >= total:
            for recipient, amount in recipients:
                self.transfer(from_addr, recipient, amount)
            return True
        return False


# Demonstrate overflow attack
print("\n" + "="*70)
print("INTEGER OVERFLOW VULNERABILITY DEMONSTRATION")
print("="*70)

token = TokenContractVulnerable()
print(f"\nInitial owner balance: {token.balances['Owner']}")
print(f"Maximum uint256 value: {token.MAX_UINT:,}")

# Craft overflow attack
# Choose two large numbers that overflow when added
amount1 = 2**255  # Half of max
amount2 = 2**255 + 1000  # Slightly more than half

print(f"\nAttempting batch transfer with amounts that overflow:")
print(f"  Amount 1: {amount1:,}")
print(f"  Amount 2: {amount2:,}")
print(f"  Sum (should be > max): {amount1 + amount2:,}")

# In Python, integers don't overflow, but simulate the behavior
simulated_overflow = (amount1 + amount2) % (2**256)
print(f"  Simulated uint256 sum (after overflow): {simulated_overflow:,}")

if simulated_overflow < 1000:
    print(f"\n💀 Overflow occurred! Total wraps to {simulated_overflow}")
    print(f"   Attacker could pass balance check and mint unlimited tokens!")


# Demonstrate secure implementation
class TokenContractSecure:
    """Token with overflow protection"""
    
    def __init__(self, supply=1000):
        self.balances = {"Owner": supply}
        
    def safe_add(self, a, b):
        """Addition with overflow check"""
        result = a + b
        if result < a:  # If result is smaller, overflow occurred
            raise OverflowError("Addition overflow detected")
        return result
    
    def safe_sub(self, a, b):
        """Subtraction with underflow check"""
        if b > a:
            raise OverflowError("Subtraction underflow detected")
        return a - b
    
    def transfer(self, from_addr, to_addr, amount):
        """Safe transfer with overflow protection"""
        try:
            from_balance = self.balances.get(from_addr, 0)
            to_balance = self.balances.get(to_addr, 0)
            
            # Use safe operations
            new_from_balance = self.safe_sub(from_balance, amount)
            new_to_balance = self.safe_add(to_balance, amount)
            
            self.balances[from_addr] = new_from_balance
            self.balances[to_addr] = new_to_balance
            return True
        except OverflowError as e:
            print(f"  ⚠️ Transfer blocked: {e}")
            return False


print("\n" + "="*70)
print("SECURE TOKEN WITH OVERFLOW PROTECTION")
print("="*70)

secure_token = TokenContractSecure()
print("\nAttempting unsafe operation on secure contract:")
success = secure_token.transfer("Owner", "Attacker", 10000)
print(f"Transfer successful: {success}")
print("\n✅ Overflow protection prevented exploitation")

### Reflection Questions (Exercise 1)

Write 200-250 words addressing:

1.  **Reentrancy Prevention**: Why is the “checks-effects-interactions”
    pattern important? How does the reentrancy guard (mutex lock)
    provide additional security even if developers follow this pattern?

2.  **Integer Overflow Evolution**: Solidity 0.8.0+ includes automatic
    overflow checks. Why did this take so long to become default? What
    are the trade-offs (gas costs, backwards compatibility, deliberate
    overflows in some algorithms)?

3.  **Security Mindset**: These vulnerabilities seem obvious in
    hindsight. Why do smart contract developers repeatedly make these
    mistakes? What systemic changes (tools, education, incentives) would
    reduce vulnerability prevalence?

## Exercise 2: DeFi Protocol Simulation and Economic Attacks

### Automated Market Maker (AMM) Implementation

In [5]:
class SimpleAMM:
    """
    Simplified Automated Market Maker (Uniswap-style)
    Constant Product Formula: x * y = k
    """
    
    def __init__(self, token_a_reserve, token_b_reserve, fee_percent=0.3):
        self.token_a = token_a_reserve
        self.token_b = token_b_reserve
        self.k = token_a_reserve * token_b_reserve
        self.fee_percent = fee_percent / 100
        self.total_liquidity_tokens = 100  # LP tokens
        self.lp_positions = {}
        self.trade_history = []
        
    def get_price(self):
        """Current price of token A in terms of token B"""
        return self.token_b / self.token_a
    
    def add_liquidity(self, user, token_a_amount, token_b_amount):
        """Add liquidity to pool"""
        # Check ratio matches current pool ratio
        current_ratio = self.token_a / self.token_b
        provided_ratio = token_a_amount / token_b_amount
        
        if abs(current_ratio - provided_ratio) / current_ratio > 0.01:
            print(f"Warning: Provided ratio doesn't match pool ratio")
        
        # Calculate LP tokens to mint (proportional to contribution)
        lp_tokens = (token_a_amount / self.token_a) * self.total_liquidity_tokens
        
        self.token_a += token_a_amount
        self.token_b += token_b_amount
        self.k = self.token_a * self.token_b
        
        self.lp_positions[user] = self.lp_positions.get(user, 0) + lp_tokens
        self.total_liquidity_tokens += lp_tokens
        
        return lp_tokens
    
    def swap_a_for_b(self, amount_a_in):
        """
        Swap token A for token B
        Formula: (x + Δx)(y - Δy) = k
        Solving for Δy: Δy = y - k/(x + Δx)
        """
        # Apply fee
        amount_a_after_fee = amount_a_in * (1 - self.fee_percent)
        
        # Calculate output amount
        new_token_a = self.token_a + amount_a_after_fee
        new_token_b = self.k / new_token_a
        amount_b_out = self.token_b - new_token_b
        
        # Price impact
        old_price = self.get_price()
        
        # Execute swap
        self.token_a = new_token_a + (amount_a_in * self.fee_percent)  # Fee goes to pool
        self.token_b = new_token_b
        
        new_price = self.get_price()
        price_impact = ((new_price - old_price) / old_price) * 100
        
        self.trade_history.append({
            'type': 'swap_a_for_b',
            'amount_in': amount_a_in,
            'amount_out': amount_b_out,
            'price_before': old_price,
            'price_after': new_price,
            'price_impact': price_impact
        })
        
        return amount_b_out, price_impact
    
    def swap_b_for_a(self, amount_b_in):
        """Swap token B for token A"""
        amount_b_after_fee = amount_b_in * (1 - self.fee_percent)
        
        new_token_b = self.token_b + amount_b_after_fee
        new_token_a = self.k / new_token_b
        amount_a_out = self.token_a - new_token_a
        
        old_price = self.get_price()
        
        self.token_b = new_token_b + (amount_b_in * self.fee_percent)
        self.token_a = new_token_a
        
        new_price = self.get_price()
        price_impact = ((new_price - old_price) / old_price) * 100
        
        self.trade_history.append({
            'type': 'swap_b_for_a',
            'amount_in': amount_b_in,
            'amount_out': amount_a_out,
            'price_before': old_price,
            'price_after': new_price,
            'price_impact': price_impact
        })
        
        return amount_a_out, price_impact
    
    def calculate_impermanent_loss(self, user, current_price_a_in_b):
        """
        Calculate impermanent loss for liquidity provider
        Compares LP value to just holding tokens
        """
        lp_tokens = self.lp_positions.get(user, 0)
        lp_share = lp_tokens / self.total_liquidity_tokens
        
        # Current LP position value
        lp_token_a = self.token_a * lp_share
        lp_token_b = self.token_b * lp_share
        lp_value = lp_token_a * current_price_a_in_b + lp_token_b
        
        # Estimate initial position (assume deposited at inception proportionally)
        initial_token_a = lp_token_a  # Simplified assumption
        initial_token_b = lp_token_b
        
        # Value if just held
        hold_value = initial_token_a * current_price_a_in_b + initial_token_b
        
        # Impermanent loss percentage
        if hold_value > 0:
            il_percent = ((lp_value - hold_value) / hold_value) * 100
        else:
            il_percent = 0
            
        return il_percent, lp_value, hold_value


# Demonstrate AMM mechanics
print("="*70)
print("AUTOMATED MARKET MAKER (AMM) SIMULATION")
print("="*70)

# Initialize AMM with ETH-USDC pool
amm = SimpleAMM(token_a_reserve=10, token_b_reserve=20000, fee_percent=0.3)

print(f"\nInitial pool state:")
print(f"  ETH (token A): {amm.token_a:.2f}")
print(f"  USDC (token B): {amm.token_b:.2f}")
print(f"  Price: {amm.get_price():.2f} USDC/ETH")
print(f"  k (constant product): {amm.k:,.0f}")

# Normal trade
print("\n" + "-"*70)
print("Normal user trade: Buy 1 ETH")
print("-"*70)

usdc_paid, impact = amm.swap_b_for_a(2100)
print(f"  USDC paid: {2100:.2f}")
print(f"  ETH received: {usdc_paid:.4f}")
print(f"  Price impact: {impact:+.2f}%")
print(f"  New price: {amm.get_price():.2f} USDC/ETH")

### Flash Loan Attack Simulation

In [6]:
class LendingProtocol:
    """Simplified lending protocol using AMM as price oracle"""
    
    def __init__(self, amm):
        self.amm = amm
        self.collateral = {}  # ETH collateral
        self.borrowed = {}    # USDC borrowed
        self.collateral_factor = 0.75  # Can borrow up to 75% of collateral value
        
    def deposit_collateral(self, user, eth_amount):
        """Deposit ETH as collateral"""
        self.collateral[user] = self.collateral[user].get(user, 0) + eth_amount
        
    def borrow(self, user, usdc_amount):
        """Borrow USDC against ETH collateral"""
        eth_collateral = self.collateral.get(user, 0)
        eth_price = self.amm.get_price()  # Get price from AMM (VULNERABILITY!)
        
        max_borrow = eth_collateral * eth_price * self.collateral_factor
        current_borrow = self.borrowed.get(user, 0)
        
        if current_borrow + usdc_amount <= max_borrow:
            self.borrowed[user] = current_borrow + usdc_amount
            return True
        return False
    
    def get_max_borrow(self, user):
        """Calculate maximum borrow capacity"""
        eth_collateral = self.collateral.get(user, 0)
        eth_price = self.amm.get_price()
        return eth_collateral * eth_price * self.collateral_factor


def simulate_flash_loan_attack():
    """
    Simulate oracle manipulation attack using flash loan
    
    Attack steps:
    1. Flash loan large amount of USDC
    2. Swap USDC for ETH on AMM (raises ETH price)
    3. Deposit ETH as collateral
    4. Borrow max USDC using inflated price
    5. Swap ETH back to USDC
    6. Repay flash loan
    7. Keep profit!
    """
    print("\n" + "="*70)
    print("FLASH LOAN ATTACK SIMULATION")
    print("="*70)
    
    # Fresh AMM and lending protocol
    amm = SimpleAMM(token_a_reserve=10, token_b_reserve=20000, fee_percent=0.3)
    lending = LendingProtocol(amm)
    
    print("\nInitial state:")
    print(f"  AMM ETH: {amm.token_a:.2f}")
    print(f"  AMM USDC: {amm.token_b:.2f}")
    print(f"  ETH price: {amm.get_price():.2f} USDC/ETH")
    
    # Step 1: Flash loan
    flash_loan_amount = 50000  # Borrow 50K USDC
    print(f"\n1. Flash loan {flash_loan_amount:,.0f} USDC")
    
    # Step 2: Manipulate price
    print(f"\n2. Swap USDC for ETH to manipulate price")
    eth_received, price_impact = amm.swap_b_for_a(flash_loan_amount)
    print(f"   Received {eth_received:.4f} ETH")
    print(f"   Price impact: {price_impact:+.2f}%")
    print(f"   New ETH price: {amm.get_price():.2f} USDC/ETH (MANIPULATED!)")
    
    # Step 3: Deposit collateral
    print(f"\n3. Deposit {eth_received:.4f} ETH as collateral")
    lending.collateral["Attacker"] = eth_received
    
    # Step 4: Borrow at manipulated price
    max_borrow = lending.get_max_borrow("Attacker")
    print(f"\n4. Borrow maximum USDC using manipulated price")
    print(f"   Max borrow at manipulated price: {max_borrow:,.2f} USDC")
    
    borrow_amount = max_borrow * 0.99  # Borrow slightly less to avoid rounding issues
    lending.borrow("Attacker", borrow_amount)
    print(f"   Borrowed: {borrow_amount:,.2f} USDC")
    
    # Step 5: Reverse swap
    print(f"\n5. Swap ETH back to USDC (reverse manipulation)")
    usdc_received, _ = amm.swap_a_for_b(eth_received * 0.99)  # Keep some ETH as collateral
    print(f"   Received {usdc_received:,.2f} USDC")
    
    # Step 6: Repay flash loan
    flash_loan_fee = flash_loan_amount * 0.0009  # 0.09% fee
    total_repay = flash_loan_amount + flash_loan_fee
    print(f"\n6. Repay flash loan + fee")
    print(f"   Repay amount: {total_repay:,.2f} USDC")
    
    # Step 7: Calculate profit
    profit = borrow_amount + usdc_received - total_repay
    print(f"\n" + "="*70)
    print(f"ATTACK RESULT")
    print(f"="*70)
    print(f"Total USDC obtained: {borrow_amount + usdc_received:,.2f}")
    print(f"Flash loan repayment: {total_repay:,.2f}")
    print(f"💀 NET PROFIT: {profit:,.2f} USDC")
    print(f"\n✋ Attack successful due to oracle manipulation!")
    print(f"   Lending protocol used AMM spot price as oracle")
    print(f"   Attacker manipulated price with flash loan capital")
    
    return profit

# Execute attack simulation
attack_profit = simulate_flash_loan_attack()

### Impermanent Loss Calculation

In [7]:
# Demonstrate impermanent loss
print("\n" + "="*70)
print("IMPERMANENT LOSS ANALYSIS")
print("="*70)

# Create AMM and add liquidity
amm_il = SimpleAMM(token_a_reserve=10, token_b_reserve=20000, fee_percent=0.3)
initial_price = amm_il.get_price()

# Alice provides liquidity
alice_eth = 1
alice_usdc = 2000
lp_tokens = amm_il.add_liquidity("Alice", alice_eth, alice_usdc)

print(f"\nAlice initial liquidity provision:")
print(f"  ETH deposited: {alice_eth:.2f}")
print(f"  USDC deposited: {alice_usdc:.2f}")
print(f"  Initial price: {initial_price:.2f} USDC/ETH")
print(f"  LP tokens received: {lp_tokens:.2f}")

# Simulate price movements
price_scenarios = [1.5, 2.0, 3.0, 4.0]

print("\n" + "-"*70)
print("Impermanent Loss at Different Prices")
print("-"*70)

il_data = []
for price_multiplier in price_scenarios:
    # Simulate price change through trades
    new_price = initial_price * price_multiplier
    
    # Calculate IL
    il_percent, lp_value, hold_value = amm_il.calculate_impermanent_loss("Alice", new_price / initial_price)
    
    il_data.append({
        'price_change': (price_multiplier - 1) * 100,
        'il_percent': il_percent,
        'lp_value': lp_value,
        'hold_value': hold_value
    })
    
    print(f"\nPrice {price_multiplier}x original ({new_price:.0f} USDC/ETH):")
    print(f"  Price change: {(price_multiplier-1)*100:+.0f}%")
    print(f"  LP position value: ${lp_value:,.2f}")
    print(f"  Just holding value: ${hold_value:,.2f}")
    print(f"  Impermanent Loss: {il_percent:.2f}%")

# Visualize IL
il_df = pd.DataFrame(il_data)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IL vs Price Change
axes[0].plot(il_df['price_change'], il_df['il_percent'], 
             marker='o', linewidth=2, markersize=8, color='red')
axes[0].axhline(0, color='black', linestyle='--', alpha=0.3)
axes[0].set_xlabel('Price Change (%)')
axes[0].set_ylabel('Impermanent Loss (%)')
axes[0].set_title('Impermanent Loss vs Price Movement', fontweight='bold')
axes[0].grid(alpha=0.3)

# Value Comparison
x_pos = np.arange(len(il_df))
width = 0.35
axes[1].bar(x_pos - width/2, il_df['lp_value'], width, 
            label='LP Position', alpha=0.8, color='blue')
axes[1].bar(x_pos + width/2, il_df['hold_value'], width, 
            label='Just Holding', alpha=0.8, color='green')
axes[1].set_xlabel('Price Scenario')
axes[1].set_ylabel('Value ($)')
axes[1].set_title('LP vs Holding Value Comparison', fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels([f"{x:+.0f}%" for x in il_df['price_change']])
axes[1].legend()
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Larger price movements create larger impermanent loss")
print("   LPs lose vs holding even when price increases!")

### Reflection Questions (Exercise 2)

Write 250-300 words addressing:

1.  **AMM Design Trade-offs**: Constant product AMMs provide
    always-available liquidity but suffer from price impact and
    impermanent loss. What alternative AMM designs exist (Curve,
    Balancer, Uniswap V3)? How do they mitigate these issues, and what
    new trade-offs do they introduce?

2.  **Oracle Security**: The flash loan attack exploited AMM spot price
    as oracle. How can protocols obtain manipulation-resistant price
    feeds? Discuss TWAPs (time-weighted average prices), Chainlink, and
    multiple-source oracles with their respective vulnerabilities and
    costs.

3.  **Flash Loan Risk-Reward**: Flash loans enable both legitimate use
    cases (arbitrage, collateral swaps) and attacks. Should DeFi
    protocols restrict flash loan access? How can protocols distinguish
    between legitimate and malicious flash loan usage?

## Exercise 3: MEV Detection and Analysis

### Simulating MEV Extraction Patterns

In [8]:
# Generate synthetic blockchain transaction data with MEV patterns
np.random.seed(42)

# Create synthetic DEX transactions
n_normal = 1000
n_sandwiches = 50

transactions = []
tx_id = 0

# Normal transactions
for i in range(n_normal):
    transactions.append({
        'tx_id': tx_id,
        'block': np.random.randint(1000, 2000),
        'tx_index': np.random.randint(0, 100),
        'address': f"user_{np.random.randint(0, 500)}",
        'type': 'swap',
        'amount_in': np.random.lognormal(3, 1),
        'amount_out': np.random.lognormal(3, 1),
        'gas_price': np.random.uniform(50, 100),
        'is_mev': False,
        'mev_type': None
    })
    tx_id += 1

# Sandwich attacks (3 transactions: front-run, victim, back-run)
for i in range(n_sandwiches):
    block_num = np.random.randint(1000, 2000)
    victim_tx_index = np.random.randint(10, 90)
    attacker = f"bot_{np.random.randint(0, 10)}"
    victim = f"user_{np.random.randint(0, 500)}"
    
    # Victim transaction (larger size)
    victim_amount = np.random.uniform(10, 50)
    
    # Front-run (buy before victim)
    transactions.append({
        'tx_id': tx_id,
        'block': block_num,
        'tx_index': victim_tx_index - 1,
        'address': attacker,
        'type': 'swap',
        'amount_in': victim_amount * 0.5,
        'amount_out': victim_amount * 0.5,
        'gas_price': np.random.uniform(150, 300),  # Higher gas
        'is_mev': True,
        'mev_type': 'sandwich_front'
    })
    tx_id += 1
    
    # Victim transaction
    transactions.append({
        'tx_id': tx_id,
        'block': block_num,
        'tx_index': victim_tx_index,
        'address': victim,
        'type': 'swap',
        'amount_in': victim_amount,
        'amount_out': victim_amount * 0.95,  # Worse price due to sandwich
        'gas_price': np.random.uniform(50, 100),
        'is_mev': False,
        'mev_type': None
    })
    tx_id += 1
    
    # Back-run (sell after victim)
    transactions.append({
        'tx_id': tx_id,
        'block': block_num,
        'tx_index': victim_tx_index + 1,
        'address': attacker,
        'type': 'swap',
        'amount_in': victim_amount * 0.5,
        'amount_out': victim_amount * 0.55,  # Profit from price impact
        'gas_price': np.random.uniform(150, 300),
        'is_mev': True,
        'mev_type': 'sandwich_back'
    })
    tx_id += 1

df = pd.DataFrame(transactions)
df = df.sort_values(['block', 'tx_index']).reset_index(drop=True)

print("="*70)
print("BLOCKCHAIN TRANSACTION DATASET WITH MEV")
print("="*70)
print(f"\nTotal transactions: {len(df):,}")
print(f"Normal transactions: {(~df['is_mev']).sum():,}")
print(f"MEV transactions: {df['is_mev'].sum():,}")
print(f"Sandwich attacks: {n_sandwiches} (3 txs each)")

print("\n" + "-"*70)
print("Sample transactions:")
print("-"*70)
print(df[['tx_id', 'block', 'tx_index', 'address', 'type', 'amount_in', 'gas_price', 'mev_type']].head(15))

### Detecting Sandwich Attacks

In [9]:
def detect_sandwich_attacks(df):
    """
    Detect sandwich attacks by finding transaction triplets:
    1. Same attacker address in positions i and i+2
    2. Different address in position i+1
    3. Transactions in consecutive positions within same block
    4. Higher gas prices for attacker transactions
    """
    sandwiches_detected = []
    
    # Group by block
    for block_num, block_df in df.groupby('block'):
        block_df = block_df.sort_values('tx_index').reset_index(drop=True)
        
        # Check consecutive triplets
        for i in range(len(block_df) - 2):
            tx1 = block_df.iloc[i]
            tx2 = block_df.iloc[i + 1]
            tx3 = block_df.iloc[i + 2]
            
            # Check sandwich pattern
            if (tx1['address'] == tx3['address'] and  # Same attacker
                tx1['address'] != tx2['address'] and  # Different victim
                tx1['tx_index'] + 1 == tx2['tx_index'] and  # Consecutive
                tx2['tx_index'] + 1 == tx3['tx_index'] and
                tx1['gas_price'] > tx2['gas_price'] + 20 and  # Higher gas
                tx3['gas_price'] > tx2['gas_price'] + 20):
                
                # Calculate profit (simplified)
                profit = (tx3['amount_out'] - tx1['amount_in']) * 2000  # Assume $2000/ETH
                
                sandwiches_detected.append({
                    'block': block_num,
                    'attacker': tx1['address'],
                    'victim': tx2['address'],
                    'front_run_tx': tx1['tx_id'],
                    'victim_tx': tx2['tx_id'],
                    'back_run_tx': tx3['tx_id'],
                    'victim_amount': tx2['amount_in'],
                    'estimated_profit': profit,
                    'victim_slippage': ((tx2['amount_in'] - tx2['amount_out']) / tx2['amount_in']) * 100
                })
    
    return pd.DataFrame(sandwiches_detected)

# Detect sandwiches
print("\n" + "="*70)
print("SANDWICH ATTACK DETECTION")
print("="*70)

sandwiches = detect_sandwich_attacks(df)

print(f"\nSandwiches detected: {len(sandwiches)}")
print(f"True sandwiches (ground truth): {n_sandwiches}")
print(f"Detection accuracy: {(len(sandwiches)/n_sandwiches)*100:.1f}%")

if len(sandwiches) > 0:
    print("\n" + "-"*70)
    print("Sample detected sandwiches:")
    print("-"*70)
    print(sandwiches[['block', 'attacker', 'victim', 'victim_amount', 
                      'estimated_profit', 'victim_slippage']].head())
    
    # Statistics
    print("\n" + "-"*70)
    print("Sandwich Attack Statistics:")
    print("-"*70)
    print(f"Total estimated MEV extracted: ${sandwiches['estimated_profit'].sum():,.0f}")
    print(f"Average profit per sandwich: ${sandwiches['estimated_profit'].mean():,.0f}")
    print(f"Average victim slippage: {sandwiches['victim_slippage'].mean():.2f}%")
    print(f"Max profit in single sandwich: ${sandwiches['estimated_profit'].max():,.0f}")
    
    # Top attackers
    print("\n" + "-"*70)
    print("Top MEV Extractors:")
    print("-"*70)
    top_attackers = sandwiches.groupby('attacker')['estimated_profit'].agg(['sum', 'count'])
    top_attackers.columns = ['Total Profit', 'Attacks']
    top_attackers = top_attackers.sort_values('Total Profit', ascending=False).head()
    print(top_attackers)

### MEV Visualization and Impact Analysis

In [10]:
# Visualize MEV patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Gas price distribution (MEV vs normal)
axes[0, 0].hist(df[~df['is_mev']]['gas_price'], bins=30, alpha=0.7, 
                label='Normal', color='blue', edgecolor='black')
axes[0, 0].hist(df[df['is_mev']]['gas_price'], bins=30, alpha=0.7, 
                label='MEV', color='red', edgecolor='black')
axes[0, 0].set_xlabel('Gas Price (Gwei)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Gas Price Distribution: MEV vs Normal Transactions', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. MEV profit over time (by block)
if len(sandwiches) > 0:
    block_mev = sandwiches.groupby('block')['estimated_profit'].sum()
    axes[0, 1].plot(block_mev.index, block_mev.values, marker='o', 
                    linewidth=1, markersize=4, color='darkred')
    axes[0, 1].set_xlabel('Block Number')
    axes[0, 1].set_ylabel('MEV Extracted ($)')
    axes[0, 1].set_title('MEV Extraction Over Time', fontweight='bold')
    axes[0, 1].grid(alpha=0.3)

# 3. Victim transaction sizes
if len(sandwiches) > 0:
    axes[1, 0].hist(sandwiches['victim_amount'], bins=20, color='orange', 
                    edgecolor='black', alpha=0.7)
    axes[1, 0].set_xlabel('Victim Transaction Size')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Distribution of Sandwich Victim Transaction Sizes', 
                         fontweight='bold')
    axes[1, 0].grid(alpha=0.3)

# 4. Profit vs Slippage
if len(sandwiches) > 0:
    axes[1, 1].scatter(sandwiches['victim_slippage'], sandwiches['estimated_profit'],
                       alpha=0.6, s=50, color='purple')
    axes[1, 1].set_xlabel('Victim Slippage (%)')
    axes[1, 1].set_ylabel('Attacker Profit ($)')
    axes[1, 1].set_title('MEV Profit vs Victim Slippage', fontweight='bold')
    axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate impact on average user
print("\n" + "="*70)
print("MEV IMPACT ANALYSIS")
print("="*70)

if len(sandwiches) > 0:
    total_victim_loss = sandwiches['estimated_profit'].sum()
    avg_loss_per_victim = sandwiches['estimated_profit'].mean()
    pct_transactions_sandwiched = (len(sandwiches) / len(df[~df['is_mev']])) * 100
    
    print(f"\nTotal value extracted from users: ${total_victim_loss:,.0f}")
    print(f"Average loss per sandwich victim: ${avg_loss_per_victim:,.0f}")
    print(f"Percentage of transactions sandwich: {pct_transactions_sandwiched:.1f}%")
    print(f"Extra slippage suffered: {sandwiches['victim_slippage'].mean():.2f}%")
    
    print("\n💡 MEV represents hidden tax on DeFi users")
    print("   Ordinary traders suffer worse execution prices")
    print("   Value transfers to sophisticated MEV extractors")

### Reflection Questions (Exercise 3)

Write 200-250 words addressing:

1.  **MEV Fairness**: MEV extraction creates hidden costs for DeFi
    users. Is this fundamentally unfair, or is it efficient market
    operation where sophisticated participants provide liquidity and
    price discovery? How does it compare to traditional finance
    practices like payment for order flow or high-frequency trading?

2.  **Detection Limitations**: Our detection method identifies sandwich
    attacks using transaction patterns. What are its limitations? Could
    attackers evade detection by splitting transactions across blocks,
    using multiple addresses, or employing more subtle strategies? How
    can detection systems stay ahead of evolving MEV tactics?

3.  **Mitigation Strategies**: Various approaches attempt to reduce MEV:
    private mempools (Flashbots), batch auctions (CowSwap), encrypted
    mempools, fair ordering protocols. Evaluate these approaches—which
    seems most promising? What are the trade-offs (efficiency,
    decentralization, complexity)?

## Summary and Integration

### What We’ve Learned

Through these exercises, you’ve:

1.  **Analyzed smart contract vulnerabilities** (reentrancy, overflow,
    access control) understanding exploit mechanics and defenses

2.  **Simulated DeFi protocol economics** (AMMs, flash loans,
    impermanent loss) revealing how automated systems work and fail

3.  **Implemented economic attacks** demonstrating how flash loans
    enable capital-efficient exploitation of protocol vulnerabilities

4.  **Detected MEV extraction patterns** quantifying the hidden tax
    sophisticated actors impose on ordinary users

5.  **Evaluated security trade-offs** understanding that no defense is
    perfect—only layers of protection reduce risk

6.  **Connected theory to practice** seeing how abstract vulnerabilities
    translate to billions in losses and hidden costs

### Connections to Course Themes

-   **Week 8 (Blockchain & Fraud)**: Smart contracts are where
    blockchain meets application logic—where innovation and most
    vulnerabilities occur

-   **Week 7 (Cryptocurrency Markets)**: DeFi protocols create new
    market structures with unique manipulation opportunities

-   **Week 6 (Financial Inclusion)**: DeFi promises democratized finance
    but complexity, MEV, and risk create barriers for ordinary users

-   **Week 3 (Platforms)**: DeFi protocols are platforms with network
    effects, governance challenges, and multisided markets

### Critical Evaluation Framework

When evaluating smart contracts and DeFi protocols:

1.  **Security analysis**: Code audits, formal verification, economic
    security analysis
2.  **Economic viability**: Are incentives aligned? Are yields
    sustainable?
3.  **Composability risks**: How do protocol interactions create
    systemic vulnerabilities?
4.  **MEV impact**: How much value is extracted from users?
5.  **User protection**: What happens when things go wrong? Insurance?
    Governance?
6.  **Regulatory fit**: How does decentralized code-is-law align with
    legal frameworks?

### Assessment Preparation

**FIN510 Coursework 2**: You can analyze DeFi protocols (yield farming
strategies, impermanent loss optimization, MEV impact on returns) using
methods from this lab.

**FIN720**: Critical evaluation of smart contract security or DeFi
sustainability makes excellent reflective analysis—assess gap between
promises (trustless, permissionless, fair) and reality (hacks, MEV,
complexity barriers).

### Further Exploration

If interested in extending your analysis:

-   **Formal verification**: Learn tools like Certora or K Framework
    proving contract correctness
-   **Advanced MEV**: Study MEV-Boost, proposer-builder separation,
    encrypted mempools
-   **Cross-protocol attacks**: Analyze how composability creates
    systemic risk
-   **Regulatory approaches**: Evaluate how different jurisdictions
    handle DeFi regulation
-   **Economic mechanism design**: Design manipulation-resistant
    protocols

------------------------------------------------------------------------

**Excellent work! You’ve analyzed smart contract security, simulated
DeFi protocols, and detected MEV patterns—connecting technical details
to financial implications and user impact.**